# Manutenzione preventiva - 

# Manutenzione preventiva - DoS Attacks

In [8]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import functions_notebook as fn
import costants as cs
import warnings
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
data_cc_1_05h= fn.open_convert_capture(".\\captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json")


data_pf_att_1m_30m_1 = fn.open_convert_capture(".\\captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")
data_pf_att_1m_30m_2 = fn.open_convert_capture(".\\captures\\captures2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")
data_pf_att_1m_30m_3 = fn.open_convert_capture(".\\captures\\captures3\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")

In [10]:
pv_cc_1_05h = fn.create_array_values(data_cc_1_05h)


pv_pf_att_1m_30m_1 = fn.create_array_values(data_pf_att_1m_30m_1)
pv_pf_att_1m_30m_2 = fn.create_array_values(data_pf_att_1m_30m_2)
pv_pf_att_1m_30m_3 = fn.create_array_values(data_pf_att_1m_30m_3)

In [11]:
print(len(cs.df_columns))

67


In [27]:
# Costruzione dataframe
# Conversione colonna timestamp a float
# Inserimento colonna inter packet arrival time
# Cattura clean 30m_1
df_cc_30m_1 = pd.DataFrame(data=pv_cc_1_05h, columns=cs.df_columns)
df_cc_30m_1['frame.time_epoch'] = df_cc_30m_1['frame.time_epoch'].astype('float')
df_cc_30m_1[cs.ipat_field_name] = df_cc_30m_1['frame.time_epoch'].diff()



# Catture pf att 1m 30m
df1_pfatt_1m_30m_1 = pd.DataFrame(pv_pf_att_1m_30m_1, columns=cs.df_columns)
df1_pfatt_1m_30m_1['frame.time_epoch'] = df1_pfatt_1m_30m_1['frame.time_epoch'].astype('float')
df1_pfatt_1m_30m_1[cs.ipat_field_name] = df1_pfatt_1m_30m_1['frame.time_epoch'].diff()
df2_pfatt_1m_30m_2 = pd.DataFrame(pv_pf_att_1m_30m_2, columns=cs.df_columns)
df2_pfatt_1m_30m_2['frame.time_epoch'] = df2_pfatt_1m_30m_2['frame.time_epoch'].astype('float')
df2_pfatt_1m_30m_2[cs.ipat_field_name] = df2_pfatt_1m_30m_2['frame.time_epoch'].diff()
df3_pfatt_1m_30m_3 = pd.DataFrame(pv_pf_att_1m_30m_3, columns=cs.df_columns)
df3_pfatt_1m_30m_3['frame.time_epoch'] = df3_pfatt_1m_30m_3['frame.time_epoch'].astype('float')
df3_pfatt_1m_30m_3[cs.ipat_field_name] = df3_pfatt_1m_30m_3['frame.time_epoch'].diff()

# Concatenazione per righe
df_pfatt_1m_30m = pd.concat([df1_pfatt_1m_30m_1, df2_pfatt_1m_30m_2, df3_pfatt_1m_30m_3], ignore_index=True)


In [28]:
df_p_cc_05h=fn.clean_df(df_cc_30m_1)
df_pfatt_1m_30m = fn.clean_df(df_pfatt_1m_30m)

AttributeError: 'float' object has no attribute 'isnumeric'

In [40]:
df_p_cc_05h
df_pfatt_1m_30m

,frame.time_epoch,eth.type,eth.len,arp.hw.type,arp.proto.type,arp.hw.size,arp.proto.size,icmp.type,icmp.code,icmp.checksum,...,eth,arp,icmp,ip,tcp,udp,mbtcp,inter.packet_arrival_time,eth.dst_int,eth.src_int
0,1.526899e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,0,NaN,553850065211,79556642095225
1,1.526899e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,0,0.009869,79556642095225,553850065211
2,1.526899e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,0,0.000843,79556642095225,553850065211
3,1.526899e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,0,0.000019,553850065211,79556642095225
4,1.526899e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,0,0.000178,79556642095225,553850065211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124230,1.533748e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,1,0.002414,52237803166,553850065211
124231,1.533748e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,0,0.215134,553850065211,52237803166
124232,1.533748e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,1,0.094448,553850065211,52237803166
124233,1.533748e+09,2048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,1,0,1,0.009739,52237803166,553850065211


In [43]:

string_columns = df_p_cc_05h.select_dtypes(include=['object']).columns

# Stampa i nomi delle colonne di tipo stringa
print("Colonne di tipo stringa:")
for col in string_columns:
    print(col)
len(string_columns)

Colonne di tipo stringa:
arp.hw.type
arp.proto.type
arp.hw.size
arp.proto.size
ip.checksum.status
tcp.checksum.status
udp.srcport
udp.dstport
udp.port
udp.length
udp.checksum
ip.flags.rb
ip.flags.df
ip.flags.mf
tcp.flags.res
tcp.flags.ae
tcp.flags.cwr
tcp.flags.urg
tcp.flags.ack
tcp.flags.push
tcp.flags.syn
tcp.flags.fin
eth
arp
icmp
ip
tcp
udp
mbtcp


29

In [42]:
imputer = SimpleImputer(strategy='mean')

# Applica l'imputer ai due DataFrame e convertili di nuovo in DataFrame pandas
df_p_cc_05h = pd.DataFrame(imputer.fit_transform(df_p_cc_05h), columns=df_p_cc_05h.columns)
df_pfatt_1m_30m = pd.DataFrame(imputer.fit_transform(df_pfatt_1m_30m), columns=df_pfatt_1m_30m.columns)

# Aggiungi le etichette dopo l'imputazione
df_p_cc_05h['label'] = 0  # Etichetta 0 per pacchetti puliti
df_pfatt_1m_30m['label'] = 1  # Etichetta 1 per pacchetti con attacchi

# Combina i due DataFrame
df_combined = pd.concat([df_p_cc_05h, df_pfatt_1m_30m], ignore_index=True)

# Separazione delle feature e delle etichette
X = df_combined.drop(columns=['label'])  # Tutte le colonne tranne 'label'
y = df_combined['label']  # Colonna etichetta

# Divisione dei dati in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Inizializzazione e addestramento del classificatore KNN
knn = KNeighborsClassifier(n_neighbors=3)  # Puoi cambiare n_neighbors per provare diverse configurazioni
knn.fit(X_train, y_train)

# Predizione sui dati di test
y_pred = knn.predict(X_test)

# Calcolo dell'accuratezza
accuracy = accuracy_score(y_test, y_pred)

# Stampa dell'accuratezza
print(f"Accuratezza del modello KNN: {accuracy:.2f}")

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '0x0800'